# 1. Build your own neural network with 3 hidden layers using pytorch (60 points)

In [25]:
#importing important libraries
import torch
import cv2
import os
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets

In [142]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the neural network architecture as a PyTorch module
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, output_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, hidden_size3)
        self.fc4 = nn.Linear(hidden_size3, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x

# Define the hyperparameters for the neural network and create an instance of the model
input_size = 20
output_size = 1
hidden_size1 = 20
hidden_size2 = 15
hidden_size3 = 10

model = NeuralNetwork(input_size, hidden_size1, hidden_size2, hidden_size3, output_size)

# Define the loss function and optimization algorithm for training the model
criterion = nn.MSELoss()  # Mean Squared Error loss
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer with learning rate 0.001

# Generate random input and output data for training the model
X = torch.randn(100, input_size)
y = torch.randn(100, output_size)

# Train the model using the input and output data for a specified number of epochs
for epoch in range(50):
    optimizer.zero_grad()  # Reset the gradients to zero
    output = model(X)  # Forward pass: compute the model output for the input data
    loss = criterion(output, y)  # Compute the loss between the predicted and actual output
    loss.backward()  # Backward pass: compute the gradients of the loss with respect to the model parameters
    optimizer.step()  # Update the model parameters based on the computed gradients
    if epoch % 10 == 0:
        print("Epoch: {}, Loss: {}".format(epoch, loss.item()))

# Use the trained neural network to make predictions on new data
X_new = torch.randn(1, input_size)
prediction = model(X_new)

print(prediction)  # Print the predicted output value


Epoch: 0, Loss: 1.0629559755325317
Epoch: 10, Loss: 1.045535683631897
Epoch: 20, Loss: 1.0259383916854858
Epoch: 30, Loss: 0.9981716871261597
Epoch: 40, Loss: 0.9576197266578674
tensor([[0.1923]], grad_fn=<AddmmBackward0>)


# 2. Train your model using cow teat datasets (30 points)

## You can resize the image to 75 * 75 *3

In [5]:
data_dir_train= "/Users/sayedraheel/Desktop/Computer Vision Assign/Train"
data_dir_test="/Users/sayedraheel/Desktop/Computer Vision Assign/Test"

In [174]:
import os
import torch
from torchvision import datasets, transforms

# Define the path to the test images
data_dir_test = "/Users/sayedraheel/Desktop/Computer Vision Assign/Test"

# Define the transforms for test images
transform_test = transforms.Compose([
    transforms.Resize((75,75)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# Create a dummy class mapping for test images
test_classes = {'test': 0}

# Create a custom dataset for test images
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, classes, transform=None):
        self.root_dir = root_dir
        self.classes = classes
        self.transform = transform
        self.images = sorted(os.listdir(self.root_dir))
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_name = self.images[idx]
        img_path = os.path.join(self.root_dir, img_name)
        image = Image.open(img_path).convert('RGB')
        label = self.classes['test']
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Create a custom dataset object for test images
test_dataset = CustomDataset(data_dir_test, test_classes, transform_test)

# Create a dataloader for test dataset
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)


In [179]:
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets

# Define the neural network architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        # Define fully connected layers
        self.fc1 = nn.Linear(75 * 75 * 3, 512) # input layer
        self.fc2 = nn.Linear(512, 256)         # hidden layer 1
        self.fc3 = nn.Linear(256, 10)          # hidden layer 2
        
        # Define activation functions
        self.sigmoid = nn.Sigmoid()
        self.logsoftmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x):
        # Reshape the input tensor
        x = x.view(-1, 75 * 75 * 3)
        
        # Apply activation functions to hidden layers
        x = self.sigmoid(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        
        # Apply LogSoftmax activation function to output layer
        x = self.logsoftmax(self.fc3(x))
        return x
    
# Define transforms for data preprocessing
transform_train = transforms.Compose([
    transforms.Resize((75, 75)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

transform_test = transforms.Compose([
    transforms.Resize((75, 75)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load the CIFAR-10 dataset
data_dir_train = "/Users/sayedraheel/Desktop/Computer Vision Assign/Train"
train_dataset = datasets.ImageFolder(data_dir_train, transform=transform_train)

# Create data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

# Create the neural network
model = Net()

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the network
epochs = 10
for epoch in range(epochs):
    train_loss = 0.0
    train_correct = 0
    
    # Iterate over the training dataset in batches
    for images, labels in train_loader:
        # Flatten the input images
        images = images.view(images.shape[0], -1)
        
        # Forward pass
        output = model(images)
        loss = criterion(output, labels)
        
        # Backward pass and optimization step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Update training loss and accuracy
        train_loss += loss.item() * images.size(0)
        _, predicted = torch.max(output.data, 1)
        train_correct += (predicted == labels).sum().item()
        
    # Compute training loss and accuracy
    train_loss /= len(train_loader.dataset)
    train_accuracy = 100. * train_correct / len(train_loader.dataset)
    
    # Print training progress
    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%")


Epoch 1/10, Train Loss: 1.1380, Train Accuracy: 54.05%
Epoch 2/10, Train Loss: 0.9891, Train Accuracy: 57.70%
Epoch 3/10, Train Loss: 0.9592, Train Accuracy: 60.40%
Epoch 4/10, Train Loss: 0.9456, Train Accuracy: 60.05%
Epoch 5/10, Train Loss: 0.9337, Train Accuracy: 59.79%
Epoch 6/10, Train Loss: 0.9335, Train Accuracy: 60.57%
Epoch 7/10, Train Loss: 0.9356, Train Accuracy: 61.36%
Epoch 8/10, Train Loss: 0.9232, Train Accuracy: 60.66%
Epoch 9/10, Train Loss: 0.9279, Train Accuracy: 60.92%
Epoch 10/10, Train Loss: 0.9244, Train Accuracy: 60.92%


# 3. Evaluate your model using the developed software (10 points)

In [180]:
# Set the model to evaluation mode
model.eval()

# Initialize variables to keep track of accuracy
test_loss = 0.0
test_correct = 0

# Disable gradient calculations
with torch.no_grad():
    for images, labels in test_loader:
        # Flatten the input images
        images = images.view(images.shape[0], -1)
        
        # Forward pass
        output = model(images)
        loss = criterion(output, labels)
        
        # Update test loss and accuracy
        test_loss += loss.item() * images.size(0)
        _, predicted = torch.max(output.data, 1)
        test_correct += (predicted == labels).sum().item()

# Compute test loss and accuracy
test_loss /= len(test_loader.dataset)
test_accuracy = 100. * test_correct / len(test_loader.dataset)

print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")

Test Loss: 1.1817, Test Accuracy: 48.42%


In [182]:
Test_predictions += predicted.tolist()
df = pd.DataFrame({'predicted': Test_predictions})
df.to_csv('test00_predictions.csv', index=False)